In [5]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import logging as lg
lg.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=lg.INFO)
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

# General column meanings
* y_org: what the orginal time series data is (should have missing values staying as missing!)
* y: what the model was trained on (the y in val section is kinda irrelevant then)
* yhat_org: original unchanged output of the models predict function
* yhat: final predictions including post processing
# Scoring
* Should all be scored using y_org vs yhat
* SHOULD MAKE SURE y_org HASN't HAD fillna(0)

# AIM: Get new smape -> v1.9
* series from pagedf into y_org

## Version 1.9
Should set version directory name in next cell. Should describe version specifics (outliers, holidays, validation period)

* TRAINING
   * Val indexing on -60
   * Linear growth
   * Fill ALL other NaNs to 0 BUT KEEP y_org (get it from pagedf)
   * Now with try:except: for the `RuntimeError': k initialized to invalid value (-nan)` which replaces first `y` with 0.001
   
* PREDICTIONS
   * Truncating predictions at 0 
   * Rounding to nearest int

### Remarks
* ?


In [7]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'

In [8]:
TARGET_VERSION = 'v1.9/'
BASE_VERSION = 'v1.6/'
os.makedirs(PROPHET_PATH+TARGET_VERSION)

In [9]:
pagedf = pd.read_feather(PROPHET_PATH+'pagedf.f')

In [10]:
forecast_files = [x.split('/')[-1] for x in glob.glob(PROPHET_PATH+BASE_VERSION+'*df.f')]

In [11]:
forecast_files[18284]

'20274df.f'

In [12]:
pagedf[forecast_files[18284][:-4]].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: 20274, dtype: float64

In [13]:
[x[:-4] for x in forecast_files[:2]]

['61780', '88262']

In [14]:
base_df = pd.read_feather(PROPHET_PATH+BASE_VERSION+forecast_files[18284])

In [15]:
base_df.head()

,ds,t,trend,seasonal_lower,seasonal_upper,trend_lower,trend_upper,yhat_lower,yhat_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,seasonal,yhat,yhat_org,y,train
0,2015-07-01,0.000000,30.150998,-6.165501,-6.165501,30.150998,30.150998,1.740480,47.605795,15.717732,15.717732,15.717732,-21.883233,-21.883233,-21.883233,-6.165501,23.985497,23.985497,0.001,1.0
1,2015-07-02,0.002045,30.433296,-11.725479,-11.725479,30.433296,30.433296,-3.186984,40.510026,11.869730,11.869730,11.869730,-23.595208,-23.595208,-23.595208,-11.725479,18.707817,18.707817,0.000,1.0
2,2015-07-03,0.004090,30.715594,-22.109723,-22.109723,30.715594,30.715594,-12.304424,30.104955,3.091730,3.091730,3.091730,-25.201454,-25.201454,-25.201454,-22.109723,8.605870,8.605870,0.000,1.0
3,2015-07-04,0.006135,30.997892,-51.952387,-51.952387,30.997892,30.997892,-43.153771,1.080831,-25.275530,-25.275530,-25.275530,-26.676857,-26.676857,-26.676857,-51.952387,0.000000,-20.954495,0.000,1.0
4,2015-07-05,0.008180,31.280190,-51.841585,-51.841585,31.280190,31.280190,-43.427529,0.711774,-23.840906,-23.840906,-23.840906,-28.000678,-28.000678,-28.000678,-51.841585,0.000000,-20.561395,0.000,1.0


In [16]:
val_results = []
for file in tqdm(forecast_files):
    base_df = pd.read_feather(PROPHET_PATH+BASE_VERSION+file)
    # y_org to be the orginal timeseries data
    base_df.loc[:,'y_org'] = pagedf[file[:-4]]
    base_df.to_feather(PROPHET_PATH+TARGET_VERSION+file)
    full_smape = wiki.val.smape(base_df.y_org, base_df.yhat)
    val_smape = wiki.val.smape(base_df[base_df['train'] == 0].y_org,base_df[base_df['train'] == 0].yhat)
    val_results.append((file[:-4], full_smape, val_smape))
val_results = pd.DataFrame(val_results, columns=['page_index',TARGET_VERSION[:-1]+'_full',TARGET_VERSION[:-1]+'_val'])
val_results.to_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')

100%|██████████| 145063/145063 [59:26<00:00, 40.67it/s] 


In [17]:
resultsdf = pd.read_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')

In [18]:
resultsdf['v1.9_val'].mean()
# base was v1.6 at 81.99

81.696788490776427